<a href="https://colab.research.google.com/github/liuchiente/machine_deep_learning/blob/main/Emotion_Classifier_with_Simple_RNN_%E4%BD%BF%E7%94%A8%E7%B0%A1%E5%96%AE%E7%9A%84RNN%E5%AF%A6%E7%8F%BE%E6%83%85%E6%84%9F%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **連接並mount 雲端硬碟**

In [ ]:
#定義路徑
SYS_DIR = "/content/drive/My Drive/Colab Notebooks/RNN_Pytorch_Online_Shopping/"
trained_Model_Path = "trained_model"
trained_Model_Name = "RNN-model.pt"

In [ ]:
#連接並mount自己的雲端硬碟(點選跑出網址複製最後的授權碼貼上並執行)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#切換執行目錄
import os
if not os.path.isdir(SYS_DIR):
    os.mkdir(SYS_DIR)
os.chdir(SYS_DIR)

In [ ]:
!ls

### **下載dataset**

In [ ]:
import os
def fileExists(filepath):
  return  os.path.exists(filepath)
#我把資料集放到我的github
if fileExists(SYS_DIR+'dataset/online_shopping_10_cats.csv')!= True:
  !wget -P dataset/ https://raw.githubusercontent.com/liuchiente/UDICatNCHU/main/online_shopping_10_cats.csv

In [ ]:
#引入套件
import pandas as pd
import numpy as np

In [ ]:
#加載資料集

pd_all = pd.read_csv(SYS_DIR + 'dataset/online_shopping_10_cats.csv')

print('評論數目（全）：%d' % pd_all.shape[0])
print('評論數目（正向）：%d' % pd_all[pd_all.label==1].shape[0])
print('評論數目（負向）：%d' % pd_all[pd_all.label==0].shape[0])
print(type(pd_all))

評論數目（全）：62774
評論數目（正向）：31728
評論數目（負向）：31046
<class 'pandas.core.frame.DataFrame'>


In [ ]:
#隨機抽樣20筆
pd_all.sample(20)

,cat,label,review
23582,水果,0,这个苹果太小了，而且硬邦邦的没有水分，是一次非常非常难忘的购物，再也不会买了
42240,衣服,1,牛仔裤大小合适，穿上很舒服，老公很喜欢，物流发货也很快，下次还会来购买！
56564,酒店,1,"地点很近沈阳北站,打的不会跳表.服务很好,尤其是前台和bell boy.标间大床房床?"
10567,平板,0,还真你码是 草，老子一直在你这买东西，还给我来个这样的，在你 吗也不从你这买东西了，就这样，...
12826,平板,0,交过钱了客服就不理人.平板，平板感觉用电很快.充电很慢
28851,洗发水,1,还没用不知道是不是正品，看起还行，，，，，
57378,酒店,1,酒店很好，装修得体，服务佳，性价比高。赠送的早餐非常丰富，晚上自助餐及其丰盛，而且品质高。前...
1828,书籍,1,不错的选择，虽然是差不多全图画的了，而且内容比较浅．但是可以很好的培养孩子们的观察＼自我阅读...
57098,酒店,1,房间设施比较旧，餐饮贵，服务不错，没有任何景观。外观比狮林好点
58220,酒店,0,"这家酒店非常差,大家不要给它的外表所欺骗,房间非常旧,而且房间烟味很重,我觉得连招待所也要比它好"


In [ ]:
#構造平衡語料
pd_positive = pd_all[pd_all.label==1]
pd_negative = pd_all[pd_all.label==0]

def get_balance_corpus(corpus_size, corpus_pos, corpus_neg):
    sample_size = corpus_size // 2
    pd_corpus_balance = pd.concat([corpus_pos.sample(sample_size, replace=corpus_pos.shape[0]<sample_size), \
                                   corpus_neg.sample(sample_size, replace=corpus_neg.shape[0]<sample_size)])
    
    print('評論數目(總體)：%d' % pd_corpus_balance.shape[0])
    print('評論數目(正向)：%d' % pd_corpus_balance[pd_corpus_balance.label==1].shape[0])
    print('評論數目(負向)：%d' % pd_corpus_balance[pd_corpus_balance.label==0].shape[0])    
    
    return pd_corpus_balance

In [ ]:
# 在訓練資料中,為了避免過度擬合, 會盡量把各類型資料做平衡
pd_60000 = get_balance_corpus(60000, pd_positive, pd_negative)

評論數目(總體)：60000
評論數目(正向)：30000
評論數目(負向)：30000


In [ ]:
#隨機抽樣20筆
pd_60000.sample(20)

,cat,label,review
8143,平板,1,非常好，很值，声音效果很牛逼，买了之后价格立马涨价178，这套路，
12863,平板,0,差评！太差，玩游戏两小时就没电。
25701,水果,0,说好的80果，结果只有一个是80的，第一次这么失望！
55802,酒店,1,酒店位置比较适中，离咱们公司客户近，开会开得晚或一大早过去开会都很方便。
30240,洗发水,1,用过之后很清爽，喜欢这个味道，洗过后头发变得比较柔软，淡淡的薄荷香味，十分舒服。
6785,平板,1,很好很实用，京东就是好
3740,书籍,0,"不错,好象更适合小一点的宝宝看,图画很简单,明了.大宝宝看有点简单了."
20701,水果,1,这次的比较小个，也没有什么特别处，一般
60322,酒店,0,到的那天 系统坏了 排队等了半小时 房间很旧 插座很少 被子有点潮 被套有几个破洞 被子掀开...
20965,水果,1,好吃，挺新鲜的，我在家门口水果摊买的都没有这么红，还比这个贵，物美价廉了 京东生鲜派送...


#數據集分割
一般我們做機器學習會將數據分為訓練集和測試集，而在深度學習中，需要多輪訓練學習，每次的學習過程都包括訓練和驗證，最後再進行測試。所以需要將數據分成訓練、驗證和測試數據。

In [ ]:
def split_csv(infile, trainfile, valtestfile, seed=999, ratio=0.2):	
    df = pd.read_csv(infile)
    df["review"] = df.review.str.replace("\n", " ")	
    idxs = np.arange(df.shape[0])	
    np.random.seed(seed)	
    np.random.shuffle(idxs)	
    val_size = int(len(idxs) * ratio)	
    df.iloc[idxs[:val_size], :].to_csv(valtestfile, index=False)	
    df.iloc[idxs[val_size:], :].to_csv(trainfile, index=False)
def split_dataFrame(df, trainfile, valtestfile, seed=999, ratio=0.2):	
    df["review"] = df.review.str.replace("\n", " ")	
    idxs = np.arange(df.shape[0])	
    np.random.seed(seed)	
    np.random.shuffle(idxs)	
    val_size = int(len(idxs) * ratio)	
    df.iloc[idxs[:val_size], :].to_csv(valtestfile, index=False)	
    df.iloc[idxs[val_size:], :].to_csv(trainfile, index=False)
#先將數據分為train.csv和test.csv	
split_dataFrame(df=pd_60000, 	
        trainfile=SYS_DIR + 'dataset/train.csv', 	
        valtestfile=SYS_DIR + 'dataset/test.csv', 	
        seed=999, 	
        ratio=0.2)
#再將train.csv分為dataset_train.csv和dataset_valid.csv
split_csv(infile=SYS_DIR + 'dataset/train.csv', 	
        trainfile=SYS_DIR + 'dataset/dataset_train.csv', 	
        valtestfile=SYS_DIR + 'dataset/dataset_valid.csv', 	
        seed=999, 	
        ratio=0.2)

In [ ]:
#安裝結巴
!pip install jieba

In [ ]:
# 因為原文是簡體中文, 所以使用 OpenCC 來達成繁簡轉換
! pip install opencc-python-reimplemented

In [ ]:
import jieba
import jieba.posseg as psg
import codecs

#利用OpenCC將簡體中文轉繁體中文
from opencc import OpenCC
cc = OpenCC('s2t') #s2t: 簡體中文 -> 繁體中文

# jieba custom setting.
jieba.set_dictionary(SYS_DIR + 'dict/dict.txt.big.txt')


# load stopwords set 引入了停用詞(停用詞就是像英文中的 the,a,this，中文的你我他，與其他詞相比顯得不怎麼重要)
stopword_set = set()
with codecs.open(SYS_DIR + 'dict/stopwords.txt','r','utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))
	
#中文斷詞
def tokenize(sentence):
    converted = cc.convert(sentence) #簡>繁
    words = jieba.posseg.cut(converted)
    words = [w.word for w in words if w.word not in stopword_set ]
    #print(words)
    return words
	
print(tokenize('傅达仁今将运行安乐死，却突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。'))

Building prefix dict from /content/drive/My Drive/Colab Notebooks/RNN_Pytorch_Online_Shopping/dict/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.u129ee828390e518a65fdc8e349e93ce4.cache
Loading model cost 1.246 seconds.
Prefix dict has been built successfully.


['傅達仁', '今', '運行', '安樂死', '卻', '突然', '爆出', '20', '年前', '遭緯', '體育臺', '封殺', '懂', '哪裏', '得罪', '電視臺']


## 安裝pytorch

In [ ]:
!pip install torch torchvision torchtext

import torch
from torch import nn
from torchtext.legacy import data,datasets

#設置固定生成隨機數的種子，使得每次運行文件時生成的隨機數相同
if torch.cuda.is_available():
    print("gpu cuda is available!")
    torch.cuda.manual_seed(1000)
else:
    print("cuda is not available! cpu is available!")
    torch.manual_seed(1000)

gpu cuda is available!


In [ ]:
CAT = data.Field(sequential=True, fix_length=20)
LABEL = data.LabelField(dtype = torch.float)	
TEXT = data.Field(tokenize = tokenize,#斷詞function 	
                      lower=True,	#是否將數據轉小寫
                      fix_length=100,	#每條數據的長度
                      stop_words=None)
train_data, valid_data, test_data = data.TabularDataset.splits(path=SYS_DIR+'dataset/', #數據所在文件夾
                                  train='dataset_train.csv', 	
                                  validation='dataset_valid.csv',	
                                  test = 'test.csv',	
                                  format='csv',
                                  skip_header=True,	
                                  fields = [('cat',CAT),('label', LABEL),('review', TEXT)])

In [ ]:
# 建立詞典
#TEXT.build_vocab(train_data, vectors="glove.6B.100d")#可以使用預訓練word vector
TEXT.build_vocab(train_data, max_size=10000)
LABEL.build_vocab(train_data)
CAT.build_vocab(train_data)

batch_size = 30
device = torch.device('cuda')
#Iterator是torchtext到模型的輸出
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size = batch_size,
    sort_key=lambda x: len(x.review),
    device=device
)

In [ ]:
#創⼀個資料夾存模型
if not os.path.isdir(trained_Model_Path):
    os.mkdir(trained_Model_Path)

In [ ]:
vocab_size=len(TEXT.vocab) #詞典大小
embedding_dim=7000 # 詞向量維度
hidden_dim=100 # 隱藏層維度
output_dim=1 # 輸出層

## 定義模型
可用torch.nn.Linear 也可用 nn.LSTM 定義你自己的神經網路

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim , output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))

In [ ]:
rnn = RNN(vocab_size, embedding_dim, hidden_dim , output_dim)
pretrained_embedding = TEXT.vocab.vectors
#優化器
optimizer = torch.optim.Adam(rnn.parameters(), lr=1e-3)
loss_func = torch.nn.BCEWithLogitsLoss().to(device) #因為我們輸出只有1個，所以選用binary cross entropy
#用gpu
rnn.to(device)

RNN(
  (embedding): Embedding(10002, 7000)
  (rnn): RNN(7000, 100)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:
#定義計算正確率
def binary_acc(preds, y):
    """
    準確率
    """
    preds = torch.round(torch.sigmoid(preds))
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc
#定義訓練方式
def train(rnn, epoch, iterator, optimizer, loss_func):

    rnn.train() # 訓練模式

    avg_acc = []
    TrainAcc = 0.0
    TrainLoss = 0.0
    
    for step,batch in enumerate(iterator):

        optimizer.zero_grad() #優化器優化之前須將梯度歸零

        # [seq, b] => [b, 1] => [b]
        pred = rnn(batch.review).squeeze(1)
    
        acc = binary_acc(pred, batch.label).item() # 取得正確率
        TrainAcc=TrainAcc+acc
        avg_acc.append(acc)

        loss = loss_func(pred, batch.label) # loss計算
        TrainLoss = TrainLoss + loss
        
        loss.backward() #反向傳遞
        optimizer.step() #通過梯度做參數更新(更新權重)
            
    TrainLoss = TrainLoss / (step+1) # epoch loss
    TrainAcc = TrainAcc / (step+1) # epoch acc    
    avg_acc = np.array(avg_acc).mean()
    print('epoch : %d ,TrainLoss: %f , TrainAcc acc: %f , avg TrainAcc acc: %f ' %(epoch+1,round(TrainLoss.item(),3), round(TrainAcc,3), avg_acc ))

    if avg_acc > 0.9:
      torch.save(rnn, SYS_DIR+trained_Model_Path+'/'+trained_Model_Name) #存模型
      #torch.save(rnn.state_dict(), SYS_DIR+trained_Model_Path+'/'+trained_Model_Name) #只存權重
      print('save done')
    
#定義評估模式  
def eval(rnn, epoch , iterator, loss_func):

    rnn.eval() # 評估模式
    
    avg_acc = []
    TestAcc = 0.0
    TestLoss = 0.0    
    
    with torch.no_grad():
        for step,batch in enumerate(iterator):
            # [b, 1] => [b]
            pred = rnn(batch.review).squeeze(1)

            acc = binary_acc(pred, batch.label).item() # 取得正確率
            TestAcc = TestAcc + acc
            avg_acc.append(acc)

            loss = loss_func(pred, batch.label) # loss計算
            TestLoss = TestLoss + loss
            
            #if step%20 == 0:print('test loss: %.4f, test acc: %.2f' %(loss, acc))

        TestLoss = TestLoss / (step+1)
        TestAcc = TestAcc / (step+1)
        avg_acc = np.array(avg_acc).mean()
        print('epoch : %d ,TestLoss: %f , TestAcc acc: %f , avg TestAcc acc: %f ' %(epoch+1,round(TestLoss.item(),3), round(TestAcc,3), avg_acc ))

def predict_sentiment(net,sentence):
    net.eval()
    # 分詞
    tokenized = tokenize(sentence)
    # sentence 的索引
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]

    tensor = torch.LongTensor(indexed).to(device)  # seq_len
    tensor = tensor.unsqueeze(1)   # seq_len * batch_size (1)

    # tensor寫text一樣的tensor
    prediction = torch.sigmoid(net(tensor))

    return prediction.item()    


In [ ]:
# 開始訓練
print('start train')
for epoch in range(10):
  train(rnn, epoch , train_iterator, optimizer, loss_func)
  eval(rnn, epoch , test_iterator, loss_func)
print('end train')
torch.save(rnn, SYS_DIR+trained_Model_Path+'/'+trained_Model_Name) #存模型

start train
epoch : 1 ,TrainLoss: 0.695000 , TrainAcc acc: 0.500000 , avg TrainAcc acc: 0.500000 
epoch : 1 ,TestLoss: 0.704000 , TestAcc acc: 0.508000 , avg TestAcc acc: 0.508417 
epoch : 2 ,TrainLoss: 0.691000 , TrainAcc acc: 0.506000 , avg TrainAcc acc: 0.506120 
epoch : 2 ,TestLoss: 0.697000 , TestAcc acc: 0.490000 , avg TestAcc acc: 0.489750 
epoch : 3 ,TrainLoss: 0.690000 , TrainAcc acc: 0.503000 , avg TrainAcc acc: 0.503073 
epoch : 3 ,TestLoss: 0.697000 , TestAcc acc: 0.509000 , avg TestAcc acc: 0.508583 
epoch : 4 ,TrainLoss: 0.689000 , TrainAcc acc: 0.506000 , avg TrainAcc acc: 0.506198 
epoch : 4 ,TestLoss: 0.698000 , TestAcc acc: 0.509000 , avg TestAcc acc: 0.508750 
epoch : 5 ,TrainLoss: 0.689000 , TrainAcc acc: 0.503000 , avg TrainAcc acc: 0.502552 
epoch : 5 ,TestLoss: 0.699000 , TestAcc acc: 0.509000 , avg TestAcc acc: 0.509000 
epoch : 6 ,TrainLoss: 0.689000 , TrainAcc acc: 0.505000 , avg TrainAcc acc: 0.505000 
epoch : 6 ,TestLoss: 0.700000 , TestAcc acc: 0.509000 , a

## 讀取模型並預測

In [ ]:
net = torch.load(SYS_DIR+trained_Model_Path+'/'+trained_Model_Name)

while True:
  user_key = input("輸入評價:")
  if(user_key=='end'):
    break
  score=predict_sentiment(net,user_key)
  if score>0.5:
    print(score,':正面')
  else:
    print(score,'負面')